In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Iterator, Tuple

In [15]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:
def build_model_frame(df):
    """
    Create a leak-free modeling frame for forecasting dwelling_starts per province.

    Parameters
    ----------
    df : pandas.DataFrame
        Must contain at least: 'quarter', 'province',
        'dwelling_starts', 'pop_change_q', 'needed_units_q'

    Returns
    -------
    model_df : pandas.DataFrame
        Long-form table with lag features, ready for modeling.
    """
    # sort to guarantee time order within each province
    df = df.sort_values(["province", "quarter"]).copy()

    # target
    df["y"] = df["dwelling_starts"]

    # one-quarter lags
    for col in ["dwelling_starts", "pop_change_q", "needed_units_q"]:
        df[f"{col}_lag1"] = df.groupby("province")[col].shift(1)

    # same-quarter-last-year lag (4 quarters back)
    df["dwelling_starts_lag4"] = df.groupby("province")["dwelling_starts"].shift(4)

    # drop rows where any lag is missing (first few quarters per province)
    feat_cols = [
        "dwelling_starts_lag1", "dwelling_starts_lag4",
        "pop_change_q_lag1", "needed_units_q_lag1"
    ]
    model_df = df.dropna(subset=["y"] + feat_cols).reset_index(drop=True)

    return model_df, feat_cols


In [23]:
df = pd.read_csv('data/housing_adequacy_dataset.csv')

model_df, feat_cols = build_model_frame(df)

In [26]:
model_df

,quarter,province,population,starts_saar,starts_saar_q,dwelling_starts,pop_change_q,needed_units_q,hai,y,dwelling_starts_lag1,pop_change_q_lag1,needed_units_q_lag1,dwelling_starts_lag4
0,1991Q1,ab,2572947.0,8.494667,2.123667,2123.666667,9805.0,3922.0,0.541475,2123.666667,2930.833333,15354.0,6141.6,5855.416667
1,1991Q2,ab,2580625.0,12.852333,3.213083,3213.083333,7678.0,3071.2,1.046198,3213.083333,2123.666667,9805.0,3922.0,5372.583333
2,1991Q3,ab,2592306.0,13.536333,3.384083,3384.083333,11681.0,4672.4,0.724271,3384.083333,3213.083333,7678.0,3071.2,3461.750000
3,1991Q4,ab,2604031.0,14.018000,3.504500,3504.500000,11725.0,4690.0,0.747228,3504.500000,3384.083333,11681.0,4672.4,2930.833333
4,1992Q1,ab,2611786.0,16.100333,4.025083,4025.083333,7755.0,3102.0,1.297577,4025.083333,3504.500000,11725.0,4690.0,2123.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,2024Q3,sk,1247868.0,5.716333,1.429083,1429.083333,9277.0,3710.8,0.385115,1429.083333,883.333333,8275.0,3310.0,1263.000000
1525,2024Q4,sk,1256983.0,4.590667,1.147667,1147.666667,9115.0,3646.0,0.314774,1147.666667,1429.083333,9277.0,3710.8,1316.833333
1526,2025Q1,sk,1261524.0,6.101667,1.525417,1525.416667,4541.0,1816.4,0.839802,1525.416667,1147.666667,9115.0,3646.0,840.583333
1527,2025Q2,sk,1264537.0,6.118667,1.529667,1529.666667,3013.0,1205.2,1.269222,1529.666667,1525.416667,4541.0,1816.4,883.333333


In [25]:
model_df.isna().sum()

quarter                  0
province                 0
population               0
starts_saar              0
starts_saar_q            0
dwelling_starts          0
pop_change_q             0
needed_units_q           0
hai                     27
y                        0
dwelling_starts_lag1     0
pop_change_q_lag1        0
needed_units_q_lag1      0
dwelling_starts_lag4     0
dtype: int64

In [28]:
model_df['quarter'] = pd.PeriodIndex(model_df['quarter'], freq='Q').to_timestamp()

In [29]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   quarter               1529 non-null   datetime64[ns]
 1   province              1529 non-null   object        
 2   population            1529 non-null   float64       
 3   starts_saar           1529 non-null   float64       
 4   starts_saar_q         1529 non-null   float64       
 5   dwelling_starts       1529 non-null   float64       
 6   pop_change_q          1529 non-null   float64       
 7   needed_units_q        1529 non-null   float64       
 8   hai                   1502 non-null   float64       
 9   y                     1529 non-null   float64       
 10  dwelling_starts_lag1  1529 non-null   float64       
 11  pop_change_q_lag1     1529 non-null   float64       
 12  needed_units_q_lag1   1529 non-null   float64       
 13  dwelling_starts_la

In [16]:
feat_cols

['dwelling_starts_lag1',
 'dwelling_starts_lag4',
 'pop_change_q_lag1',
 'needed_units_q_lag1']

## Data Splitting

In [30]:
# checking if the 'quarter' is in datetime format
if model_df["quarter"].dtype == "O":  # strings like "1991Q1"
    model_df["quarter"] = pd.PeriodIndex(model_df["quarter"], freq="Q").to_timestamp()

In [32]:
#Chronological split (single train/test cut)
def chrono_split_q(df: pd.DataFrame, cutoff) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    cutoff: str or Timestamp (e.g., '2023-12-31' or pd.Timestamp('2023-12-31'))
    Returns train (<= cutoff) and test (> cutoff).
    """
    cutoff = pd.Timestamp(cutoff)
    tr = df[df["quarter"] <= cutoff].copy()
    te = df[df["quarter"] >  cutoff].copy()
    return tr, te

#Rolling-origin (expanding window) for quarterly panels
def rolling_origin_q(df: pd.DataFrame,
                     initial=None,
                     step: int = 1,
                     fh: int = 2) -> Iterator[Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Expanding-window backtest.
      - initial: first train end date (Timestamp or str). If None, uses 60% of timeline.
      - step: how many quarters to advance the train end each fold.
      - fh: forecast horizon in quarters (size of the test slice each fold).
    Yields (train_df, test_df) pairs over the whole *panel* (all provinces).
    """
    dates = np.sort(df["quarter"].unique())
    if initial is None:
        initial = dates[int(0.6 * len(dates))]
    else:
        initial = pd.Timestamp(initial)

    # find index of initial in the date array
    # (if not exact, snap to the nearest past available quarter)
    start_idx = np.searchsorted(dates, initial, side="right") - 1
    start_idx = max(start_idx, 0)

    for i in range(start_idx, len(dates) - fh, step):
        train_end = dates[i]
        test_slice = dates[i+1 : i+1+fh]
        tr = df[df["quarter"] <= train_end].copy()
        te = df[df["quarter"].isin(test_slice)].copy()
        if not te.empty:
            yield tr, te

## Modeling the baseline

In [34]:
# Baseline: same quarter last year
def _seasonal_naive_q(s: pd.Series, season: int = 4) -> pd.Series:
    # shift by one season (4 quarters)
    return s.shift(season)

def seasonal_naive_predict_q(train: pd.DataFrame,
                             test: pd.DataFrame,
                             target: str = "y",
                             season: int = 4) -> pd.Series:
    """
    Predict each test row as the value from the same quarter last year,
    computed per-province, using train history only.
    """
    preds = []
    for prov, gte in test.groupby("province"):
        gtr = train[train["province"] == prov]
        # stack so the shift sees the last 4 quarters of history
        hist_future = pd.concat([gtr, gte], axis=0).sort_values("quarter")
        yhat = _seasonal_naive_q(hist_future[target], season=season).loc[gte.index]
        preds.append(yhat)
    return pd.concat(preds).sort_index()

## Metrics

In [48]:
#For MAE and RMSE: lower is better. RMSE punishes big misses.
def mae(y, yhat):
    y, yhat = np.asarray(y), np.asarray(yhat)
    return np.mean(np.abs(y - yhat))

def rmse(y, yhat):
    y, yhat = np.asarray(y), np.asarray(yhat)
    return np.sqrt(np.mean((y - yhat)**2))

#MAPE/sMAPE: symmetric
def mape(y, yhat, eps=1e-8):
    """Safe MAPE (avoids divide-by-zero by flooring |y|)."""
    y, yhat = np.asarray(y), np.asarray(yhat)
    denom = np.maximum(np.abs(y), eps)
    return 100.0 * np.mean(np.abs((y - yhat) / denom))

def smape(y, yhat, eps=1e-8):
    """Symmetric MAPE."""
    y, yhat = np.asarray(y), np.asarray(yhat)
    denom = np.maximum((np.abs(y) + np.abs(yhat)) / 2.0, eps)
    return 100.0 * np.mean(np.abs(y - yhat) / denom)

#MASE: < 1 means beating the seasonal naïve
def mase(y_true, y_pred, y_train, seasonality=1, eps=1e-8):
    """
    Mean Absolute Scaled Error: MAE(model) / MAE(naive seasonal) computed on TRAIN.
    y_train is the in-sample series used to compute naive seasonal errors.
    """
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    y_train = np.asarray(y_train)

    # in-sample seasonal naive errors (needs at least seasonality+1 points)
    if len(y_train) <= seasonality:
        return np.nan

    insample_errors = np.abs(y_train[seasonality:] - y_train[:-seasonality])
    d = np.mean(insample_errors)
    d = max(d, eps)  # avoid divide-by-zero

    return np.mean(np.abs(y_true - y_pred)) / d


In [35]:
'''# RMSE: penalizes large errors
def rmse(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

# sMAPE: scale-free, symmetric between over/under-prediction; allows fair aggregation across provinces with different scales
def smape(y, yhat):
    denom = np.abs(y) + np.abs(yhat)
    denom[denom == 0.0] = 1.0
    return 200.0 * np.mean(np.abs(y - yhat) / denom)



MASE (seasonal): mean absolute error scaled by the in-sample seasonal naive MAE (using train only).
Interpretation:
MASE ≈ 1: you’re about as good as seasonal naive.
< 1: better than seasonal naive (good).
> 1: worse than seasonal naive.

Seasonal in-sample scaling: makes comparability robust across series and avoids data leakage


def mase_quarterly(y_test, yhat, y_insample, season=4):
    insample_err = np.abs(y_insample[season:] - y_insample[:-season]).mean()
    if insample_err == 0 or np.isnan(insample_err):
        return np.nan
    return np.mean(np.abs(y_test.values - yhat.values)) / insample_err
'''

In [42]:
'''# Per horizon metric table

def metric_table_q(test_df, yhat, target="y", season=4, train_df=None):
    out = test_df[["province","quarter",target]].copy()
    out["yhat"] = yhat.values
    # Rank quarters within each province's TEST slice to get forecast horizon h = 1,2,...
    out["h"] = out.groupby("province")["quarter"].rank(method="first").astype(int)

    # Aggregate by horizon across provinces
    g = out.groupby("h")
    per_h = pd.DataFrame({
        "RMSE": g.apply(lambda s: rmse(s[target], s["yhat"]), include_groups = False),
        "sMAPE": g.apply(lambda s: smape(s[target], s["yhat"]), include_groups = False)
    }).reset_index()

    # Optional province-averaged MASE (needs train_df for scaling)
    mase_val = np.nan
    if train_df is not None:
        mases = []
        for prov, gte in out.groupby("province"):
            gtr = train_df[train_df["province"] == prov]
            if len(gtr) >= season + 1 and not gte.empty:
                mases.append(mase_quarterly(gte[target], gte["yhat"], gtr[target], season=season))
        mase_val = np.nanmean(mases) if mases else np.nan

    return per_h, mase_val
'''

In [45]:
# one-off chronological split
train, test = chrono_split_q(model_df, cutoff="2023-12-31")

# rolling backtest generator
ro = rolling_origin_q(model_df, initial=None, step=1, fh=2)
first_train, first_test = next(ro)
first_train.head(), first_test.head()


(     quarter province  population  starts_saar  starts_saar_q  \
 0 1991-01-01       ab   2572947.0     8.494667       2.123667   
 1 1991-04-01       ab   2580625.0    12.852333       3.213083   
 2 1991-07-01       ab   2592306.0    13.536333       3.384083   
 3 1991-10-01       ab   2604031.0    14.018000       3.504500   
 4 1992-01-01       ab   2611786.0    16.100333       4.025083   
 
    dwelling_starts  pop_change_q  needed_units_q       hai            y  \
 0      2123.666667        9805.0          3922.0  0.541475  2123.666667   
 1      3213.083333        7678.0          3071.2  1.046198  3213.083333   
 2      3384.083333       11681.0          4672.4  0.724271  3384.083333   
 3      3504.500000       11725.0          4690.0  0.747228  3504.500000   
 4      4025.083333        7755.0          3102.0  1.297577  4025.083333   
 
    dwelling_starts_lag1  pop_change_q_lag1  needed_units_q_lag1  \
 0           2930.833333            15354.0               6141.6   
 1      

In [53]:
#train, test = chrono_split_q(model_df, cutoff="2023-12-31")

yhat_naive = seasonal_naive_predict_q(train, test, target="y", season=4)

per_h, mase_avg = metric_table_q(test, yhat_naive, target="y", season=4, train_df=train)


/var/folders/h4/8_8xkn4579zbrdb79s7zrtl00000gp/T/ipykernel_7631/3398096598.py:24: RuntimeWarning: Mean of empty slice
  mase_val = np.nanmean(mases) if mases else np.nan


In [54]:
print("Per-horizon metrics:")
display(per_h)
print(f"\nAverage MASE (quarterly): {mase_avg:.3f}")

Per-horizon metrics:


,h,RMSE,sMAPE
0,1,2183.240200,32.277738
1,2,2692.653611,27.457065
2,3,1972.167561,17.060628
3,4,2081.412930,27.983479
4,5,3023.271105,29.076095
5,6,3088.977427,19.116546
6,7,3154.124763,24.375103



Average MASE (quarterly): nan


In [49]:
yhat_naive

132      7050.416667
133      7570.000000
134     10658.250000
135     10606.416667
136     10934.833333
            ...     
1524     1263.000000
1525     1316.833333
1526      840.583333
1527      883.333333
1528     1429.083333
Name: y, Length: 77, dtype: float64

In [52]:
model_df['y'][132:]

132     10934.833333
133     11410.250000
134     12168.916667
135     13291.166667
136     13085.666667
            ...     
1524     1429.083333
1525     1147.666667
1526     1525.416667
1527     1529.666667
1528     1252.250000
Name: y, Length: 1397, dtype: float64